Import the necessary libraries

In [1]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip


In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
import plotly.graph_objects as go
import pandas_profiling
from pandas_profiling import ProfileReport
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tkinter
import seaborn as sns



Read the data with the help of pandas

In [3]:
Netflix_data = pd.read_csv('NFLX_24may2002to17apr2022.csv')


Preliminary parts to understand the data

In [4]:
Netflix_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5009 entries, 0 to 5008
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       5009 non-null   object 
 1   Open       5009 non-null   float64
 2   High       5009 non-null   float64
 3   Low        5009 non-null   float64
 4   Close      5009 non-null   float64
 5   Adj Close  5009 non-null   float64
 6   Volume     5009 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 274.1+ KB


In [5]:
print(Netflix_data.shape)

(5009, 7)


In [6]:
Netflix_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-05-24,1.214286,1.225000,1.197143,1.210000,1.210000,11104800
1,2002-05-28,1.213571,1.232143,1.157143,1.157143,1.157143,6609400
2,2002-05-29,1.164286,1.164286,1.085714,1.103571,1.103571,6757800
3,2002-05-30,1.107857,1.107857,1.071429,1.071429,1.071429,10154200
4,2002-05-31,1.078571,1.078571,1.071429,1.076429,1.076429,8464400


In [7]:
Netflix_data.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

Understand the number of missing values in each column by the number of them and the percentage

In [8]:
Netflix_data.isna().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

Understand data in the table

In [9]:
Netflix_data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,5009.000000,5009.000000,5009.000000,5009.000000,5009.000000,5.009000e+03
mean,115.912136,117.656523,114.054812,115.903179,115.903179,1.649836e+07
std,169.147478,171.543870,166.531788,169.074697,169.074697,1.918532e+07
min,0.377857,0.410714,0.346429,0.372857,0.372857,2.856000e+05
25%,3.947143,4.025714,3.871429,3.954286,3.954286,6.125900e+06
50%,25.465714,25.834286,24.971430,25.412857,25.412857,1.056510e+07
75%,147.339996,148.289993,146.220001,147.809998,147.809998,1.986530e+07
max,692.349976,700.989990,686.090027,691.690002,691.690002,3.234140e+08


In [10]:
report = ProfileReport(Netflix_data, title='Pandas Profiling Report Task B dataset CW2', html={'style':{'full_width':True}})
report.to_file(output_file="CW2_DataB_statistics.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
Netflix_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-05-24,1.214286,1.225000,1.197143,1.210000,1.210000,11104800
1,2002-05-28,1.213571,1.232143,1.157143,1.157143,1.157143,6609400
2,2002-05-29,1.164286,1.164286,1.085714,1.103571,1.103571,6757800
3,2002-05-30,1.107857,1.107857,1.071429,1.071429,1.071429,10154200
4,2002-05-31,1.078571,1.078571,1.071429,1.076429,1.076429,8464400
...,...,...,...,...,...,...,...
5004,2022-04-08,361.959991,362.500000,354.869995,355.880005,355.880005,4020500
5005,2022-04-11,350.000000,354.779999,345.200012,348.000000,348.000000,3777100
5006,2022-04-12,355.910004,359.410004,342.250000,344.100006,344.100006,3824300
5007,2022-04-13,343.920013,352.000000,341.160004,350.429993,350.429993,3231000


Choose only the close column for our prediction

In [12]:
Netflix_data =  Netflix_data[['Date','Close']]
Netflix_data.head()

,Date,Close
0,2002-05-24,1.210000
1,2002-05-28,1.157143
2,2002-05-29,1.103571
3,2002-05-30,1.071429
4,2002-05-31,1.076429


Visualize the data set

In [13]:
Netflix_data.Date=pd.to_datetime(Netflix_data.Date, format='%Y/%m/%d')
Netflix_data= Netflix_data.set_index('Date')


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Define figure for the stock prize

In [14]:
plt.clf()
plt.plot(Netflix_data.loc['2021-04-16':'2022-04-17'])
plt.savefig('CW2_partB_fig.png', dpi=1050)
plt.show

<function matplotlib.pyplot.show>

fix random seed for reproducibility

In [15]:
np.random.seed(7)

In [16]:
Netflix_data.shape

(5009, 1)

Normalize the dataset

In [17]:
Netflix_data=Netflix_data.loc['2016-04-16':'2022-04-17']
scaler = MinMaxScaler(feature_range=(0, 1))
Netflix_data_for_LSTM = scaler.fit_transform(Netflix_data)
Netflix_data.shape


(1511, 1)

Split into train and test sets

In [18]:
train_size = int(len(Netflix_data_for_LSTM) * 0.70)
test_size = len(Netflix_data_for_LSTM) - train_size
train, test = Netflix_data_for_LSTM[0:train_size, :], Netflix_data_for_LSTM[train_size:len(Netflix_data_for_LSTM), :]
train.shape

(1057, 1)

In [19]:
test.shape

(454, 1)

Convert an array of values into a dataset matrix

In [20]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)


np.random.seed(7)


Reshape into X=t and Y=t+1

In [32]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(1055, 1)
(1055,)
(452, 1)
(452,)


Reshape input to be [samples, time steps, features]

In [22]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


Create and fit the LSTM network

In [23]:
model = Sequential()
model.add(LSTM(5,input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
1055/1055 - 4s - loss: 0.0097 - 4s/epoch - 4ms/step
Epoch 2/100
1055/1055 - 2s - loss: 2.0038e-04 - 2s/epoch - 2ms/step
Epoch 3/100
1055/1055 - 2s - loss: 1.5822e-04 - 2s/epoch - 2ms/step
Epoch 4/100
1055/1055 - 2s - loss: 1.5786e-04 - 2s/epoch - 2ms/step
Epoch 5/100
1055/1055 - 2s - loss: 1.6048e-04 - 2s/epoch - 2ms/step
Epoch 6/100
1055/1055 - 2s - loss: 1.5901e-04 - 2s/epoch - 2ms/step
Epoch 7/100
1055/1055 - 2s - loss: 1.5475e-04 - 2s/epoch - 2ms/step
Epoch 8/100
1055/1055 - 2s - loss: 1.6167e-04 - 2s/epoch - 2ms/step
Epoch 9/100
1055/1055 - 2s - loss: 1.5625e-04 - 2s/epoch - 2ms/step
Epoch 10/100
1055/1055 - 2s - loss: 1.5528e-04 - 2s/epoch - 2ms/step
Epoch 11/100
1055/1055 - 2s - loss: 1.5231e-04 - 2s/epoch - 2ms/step
Epoch 12/100
1055/1055 - 2s - loss: 1.5576e-04 - 2s/epoch - 2ms/step
Epoch 13/100
1055/1055 - 2s - loss: 1.6402e-04 - 2s/epoch - 2ms/step
Epoch 14/100
1055/1055 - 2s - loss: 1.5507e-04 - 2s/epoch - 2ms/step
Epoch 15/100
1055/1055 - 2s - loss: 1.5492e-04 

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5)                 140       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 146
Trainable params: 146
Non-trainable params: 0
_________________________________________________________________


Make predictions

In [24]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

Invert predictions

In [25]:
trainPredict=scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

Calculate root mean squared error

In [26]:
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:, 0]))
print('Train Score: %.2f RMSE' % trainScore)
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:, 0]))
print('Test Score: %.2f RMSE' % testScore)

Train Score: 7.28 RMSE
Test Score: 23.36 RMSE


Shift train predictions for plotting

In [27]:
trainPredictPlot = np.empty_like(Netflix_data_for_LSTM)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict) + look_back, :] = trainPredict

Shift test predictions for plotting

In [28]:
testPredictPlot = np.empty_like(Netflix_data_for_LSTM)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict) + (look_back * 2) + 1:len(Netflix_data_for_LSTM) - 1, :] = testPredict

Plot baseline and predictions

In [29]:
plt.clf()
plt.plot(scaler.inverse_transform(Netflix_data_for_LSTM))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()
plt.savefig('CW2_prediction.png', dpi=1050)

Prediction price and actual price in a data frame together

In [30]:
y_actual=np.append(trainY,testY)
y_predicted=np.append(trainPredict,testPredict)
results=pd.DataFrame()
results['Actual_value']=y_actual
results['Predicted_value']=y_predicted 
results

,Actual_value,Predicted_value
0,94.339996,108.278076
1,96.769997,94.755486
2,94.980003,97.083702
3,95.900002,95.368309
4,93.559998,96.249710
...,...,...
1502,362.149994,366.186859
1503,355.880005,360.169769
1504,348.000000,354.065155
1505,344.100006,346.366455
